# 대화형 검색


In [ ]:
!pip install langchain
!pip install langchain_community
!pip install boto3
!pip install opensearch-py

# 컨텍스트 없이 물어보기


In [ ]:
query_text = "겨울에 가족과 함께 가기 좋은 숙소"

In [ ]:
from langchain_community.chat_models import BedrockChat
from langchain_core.messages import HumanMessage

model_kwargs = {  # anthropic
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 2048,
    "temperature": 0,
}

llm = BedrockChat(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",  # 파운데이션 모델 지정
    model_kwargs=model_kwargs,
)  # Claude 속성 구성

messages = [HumanMessage(content=query_text)]

print(textwrap.fill(llm(messages).content, 80))

겨울에 가족과 함께 즐길 수 있는 숙소로는 다음과 같은 곳들이 좋습니다.  1. 스키리조트 콘도/펜션 - 스키를 즐기며 가족과 함께 겨울
액티비티를 만끽할 수 있습니다. - 대표적인 곳으로는 휘닉스파크, 하이원리조트, 비비정선 등이 있습니다.  2. 온천/스파 리조트 - 따뜻한
온천수에 몸을 담그며 겨울철 추위를 녹일 수 있습니다. - 일상에서 벗어나 휴식을 취할 수 있는 곳입니다. - 대표적인 곳으로는 웰리힐리파크,
엘리시안 강촌, 설렘온천 등이 있습니다.  3. 전원주택/펜션 - 아늑하고 정겨운 분위기에서 가족들과 시간을 보낼 수 있습니다.  - 겨울
풍경을 감상하며 휴식을 취할 수 있습니다. - 대표적인 곳으로는 남이섬, 가평, 강화도 등 전원주택 단지가 있습니다.  4. 키즈 캐릭터
호텔/리조트 - 아이들이 좋아하는 캐릭터 테마로 꾸며져 있어 아이들과 함께 즐길 수 있습니다. - 실내 워터파크, 키즈룸 등 다양한 시설이
있습니다. - 대표적인 곳으로는 애니메이션호텔, 쉐라톤 그랜드 워커힐 등이 있습니다.  가족 구성원 모두가 만족할 수 있는 숙소를 선택하여 알찬
겨울 휴가를 보내시기 바랍니다.


## RAG을 통한 대화형 검색


### LangChain 구성하기


In [230]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
from langchain.vectorstores import OpenSearchVectorSearch
import boto3
import json
import textwrap

In [2]:
credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, region="us-east-1", service="es")

aos_host = "search-bedrock-opensearch-6gczufsfv5pp76bvtsd62qkrxu.aos.us-east-1.on.aws"
opensearch_auth = ("admin", "Make1StepForward!")

aos_client = OpenSearch(
    hosts=[{"host": aos_host, "port": 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=600,
)

In [3]:
import numpy as np
from langchain_community.embeddings import BedrockEmbeddings
from requests_aws4auth import AWS4Auth
from typing import Any, Dict, Iterable, List, Optional, Tuple, Callable
from typing import Type

aos_url = "https://" + aos_host

embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1")

# this is just an example, you would need to change these values to point to another opensearch instance
docsearch = OpenSearchVectorSearch(
    index_name="hotels_index",
    embedding_function=embeddings,
    opensearch_url=aos_url,
    http_auth=auth,
    connection_class=RequestsHttpConnection,
)


class SimiliarOpenSearchVectorSearch(OpenSearchVectorSearch):

    def relevance_score(self, distance: float) -> float:
        return distance

    def _select_relevance_score_fn(self) -> Callable[[float], float]:
        return self.relevance_score


open_search_vector_store = SimiliarOpenSearchVectorSearch(
    index_name="hotels_index",
    embedding_function=embeddings,
    opensearch_url=aos_url,
    http_auth=auth,
    connection_class=RequestsHttpConnection,
)

In [95]:
# you can specify custom field names to match the fields you're using to store your embedding, document text value, and metadata
docs = open_search_vector_store.similarity_search_with_score(
    query_text,
    search_type="approximate_search",
    space_type="cosinesimil",
    k=10,
)

# docs_ = open_search_vector_store.similarity_search_with_score(question, k=5)

print("found document number:" + str(len(docs)))

print("opensearch results:\n")
for doc in docs:
    print(doc)
    print("\n-----------------")

found document number:10
opensearch results:

(Document(page_content='오핑 펜션은 양양에 위치한 3성급 펜션입니다. 설악해변과 낙산사 인근에 있으며, 주방과 TV, 무선 인터넷 등의 시설을 갖추고 있습니다. 주변에는 설악해변, 낙산해변, 의상대, 낙산사 등의 관광지가 있고, 가장 가까운 공항은 양양국제공항입니다.', metadata={'address': '강원도 양양군 강현면 강현면 드윈나루2길 24-1', 'city': '양양', 'vector_field': [0.0026550293, -0.103027344, -0.06298828, 0.34179688, 0.15820312, 0.0053100586, -0.036132812, 0.00021648407, -0.34570312, -0.3828125, 0.36132812, -0.26953125, 0.020141602, 0.51953125, 0.5234375, 0.19140625, 0.14257812, -0.44921875, -0.19433594, 0.25, 0.06347656, -0.38867188, -0.33007812, 0.65625, 0.21679688, 0.35546875, 0.1953125, 0.32617188, 0.19726562, -0.21289062, -0.67578125, 0.57421875, 0.18652344, -0.81640625, -0.1796875, 0.45898438, -0.17578125, -0.35742188, 0.2421875, -0.13867188, 0.09033203, 0.100097656, 0.6953125, -0.26171875, 0.66015625, 0.17480469, 0.010314941, 0.7421875, 0.265625, 0.20898438, 0.12451172, 0.061035156, -0.39648438, -0.34179688, -0.12988281, -0.51953125, -0.44335938, -0.625

In [107]:
retriever = open_search_vector_store.as_retriever(
    # search_type="similarity_score_threshold",
    search_kwargs={"k": 10, "lambda_mult": 0.25},
)

In [108]:
query = query_text
relevant_documents = retriever.get_relevant_documents(query)

for doc in relevant_documents:
    print(doc.page_content)

오핑 펜션은 양양에 위치한 3성급 펜션입니다. 설악해변과 낙산사 인근에 있으며, 주방과 TV, 무선 인터넷 등의 시설을 갖추고 있습니다. 주변에는 설악해변, 낙산해변, 의상대, 낙산사 등의 관광지가 있고, 가장 가까운 공항은 양양국제공항입니다.
경주 웨스트 남산 한옥 펜션은 경주에 위치해 있으며 경주국립공원, 포석정, 보문호수, 황리단길 등 주요 관광지와 가까운 거리에 있습니다. 한옥 스타일의 숙소에서 주방, 전자레인지, LCD TV, 무료 와이파이 등의 시설을 갖추고 있습니다. 가까운 공항으로는 울산공항, 포항공항이 있습니다.
태안 웨스트윈드 펜션은 충청남도 태안군 안면읍 백사장1길에 위치해 있습니다. 이 2성급 펜션은 삼봉해수욕장, 간월암, 몽산포해수욕장, 안면읍, 태안해안국립공원, 오케이드타운 수목원, 부석사, 연포해수욕장, 태안해안국립공원, 동남볼링센터, 해미읍성, 서광사, 한국전통건축박물관, 삽시도, 수덕사 등 주요 관광지와 가깝습니다. 가장 가까운 공항은 청주국제공항으로 105km/65.3마일 거리에 있습니다.
경상북도 경주시에 위치한 윈드플라워는 경주 세계유산에서 10km 거리에 있으며 무료 WiFi, 바베큐 시설, 정원 및 무료 개인 주차장을 갖추고 있습니다. 객실에는 에어컨, 완비된 주방, 평면 TV, 샤워 시설과 헤어드라이어가 있는 개인 욕실, 정원 전망의 테라스가 있습니다. 인근에 석굴암(23km), 첨성대(2.4km)가 있으며 가장 가까운 공항은 포항국제공항(27km)입니다.
경주 와우 하우스 펜션은 경주에 위치한 3성급 펜션입니다. 경주 컨트리클럽, 보문호수, 블루원워터파크 등 주요 관광지와 가까운 거리에 있으며 주방, 냉장고, 스토브 등의 시설을 갖추고 있습니다. 울산공항과 포항공항에서 가깝습니다.
돌배펜션은 평창군 스윌바위길 71에 위치한 3성급 펜션입니다. 정원, 발코니, 무료 WiFi, 에어컨 객실을 제공하며 주방, 마이크로웨이브, TV, 샤워실 등 편의시설이 갖추어져 있습니다. 인근에 이효석문학관, 횡계리계곡 등의 관광지가 있고

In [214]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(memory_key="chat_history", k=10, return_messages=True)

In [313]:
from langchain import PromptTemplate

template = """

You are a korean tour guide who are experts in Korean hotels. Please answer the <QUESTION>.
Always say the hotel name, not 'this hotel' or 'that hotel'.

<CONTEXT>
{context}
</CONTEXT>

<QUESTION>
{question}
</QUESTION>
"""

prompt_template = PromptTemplate(input_variables=["question", "context"], template=template)

In [314]:
condense_template = """
Generate one standalone question based on the instructions.

<instrunctions>
- You will be given the following conversation between <chat-history> and </chat-history>
- You will be given the following follow up question between <follow-up-question> and </follow-up-question>
- Standalone question should have summary of the previous questions and answers.
</instructions>

<chat-history>
{chat_history}
</chat-history>

<follow-up-question>
{question}
</follow-up-question>

standalone question:
"""

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(condense_template)

In [326]:
from langchain.chains import ConversationalRetrievalChain

memory.clear()

conversation_with_retrieval = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": prompt_template},
    condense_question_prompt=CONDENSE_QUESTION_PROMPT,
)

chat_response = conversation_with_retrieval.invoke({"question": query_text})

print(textwrap.fill(chat_response["answer"], 80))

겨울에 가족과 함께 가기 좋은 숙소로 봄 여름 가을 겨울 캐러반을 추천합니다. 봄 여름 가을 겨울 캐러반은 강원도 평창군 봉평면에 위치한 2성급
캠핑장으로, 주방, 무료 인터넷, 냉장고, 하우스키핑 등의 시설을 갖추고 있어 가족 단위 여행객들에게 편리합니다. 또한 인근에 이효석문학관,
평창무예아트센터, 바론골, 휘닉스파크 스키리조트 등의 관광지가 있어 다양한 겨울 액티비티를 즐길 수 있습니다.


In [327]:
chat_response = conversation_with_retrieval.invoke(
    {"question": "봄 여름 가을 겨울 캐러반 호텔에 주방이 있어?"}
)

print(textwrap.fill(chat_response["answer"], 80))

네, 봄 여름 가을 겨울 캐러반에는 주방 시설이 있습니다. 질문에서 언급된 대로 "봄 여름 가을 겨울 캐러반은 주방, 무료 인터넷, 냉장고,
하우스키핑 등의 시설을 갖추고 있어" 주방 시설이 갖추어져 있습니다. 따라서 가족 단위 겨울 여행에 적합한 숙소라고 할 수 있겠습니다.


In [328]:
chat_response = conversation_with_retrieval.invoke({"question": "주변 관광지는 어디야?"})

print(textwrap.fill(chat_response["answer"], 80))

봄 여름 가을 겨울 캐러반 주변에는 이효석문학관, 평창무예아트센터, 바론골, 휘닉스파크 스키리조트 등의 관광지가 있습니다. 특히 휘닉스파크 스키리조트는 겨울 스키와 눈놀이를 즐기기에 좋은 곳입니다. 가족 단위 겨울 여행객들에게 이 숙소는 편리한 위치에 있어 적합할 것 같습니다.
